In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
import lightgbm as lgm
import xgboost as xgb

In [53]:
#!pip install lightgbm
#!pip install xgboost

In [3]:
# Read the preprocessed CSV file
dataset = pd.read_csv('preprocessed.csv')


In [4]:
dataset

,min_sip,min_lumpsum,expense_ratio,fund_size_cr,fund_age_yr,risk_level,rating,returns_1yr,returns_3yr,returns_5yr,sortino,alpha,sd,beta,sharpe
0,100,100,0.27,10,10,3,3,4.0,6.5,6.9,0.32,2.24,9.39,0.01,0.24
1,1000,1000,0.36,4288,10,1,3,5.6,4.8,5.5,1.33,1.53,0.72,0.56,1.1
2,1000,1000,0.53,157,10,5,3,2.0,18.9,9.7,3.44,2.67,10.58,0.67,1.42
3,500,1000,0.76,637,4,6,2,-0.7,17.1,0.0,2.18,-6.37,14.99,0.85,0.9
4,100,100,0.61,6386,10,6,4,4.5,18.6,9.7,3.69,1.99,10.38,0.68,1.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,500,500,0.35,1238,5,1,3,5.6,4.6,5.4,1.08,1.45,0.72,0.64,0.82
145,500,500,0.33,2463,10,2,3,4.0,6.4,6.9,1.59,4.78,1.87,2.4,0.84
146,500,500,0.39,333,10,3,3,3.8,6.1,4.9,0.98,1.17,2.36,1.02,0.62
147,500,500,0.25,2065,5,3,2,2.6,5.6,0.0,0.59,0.76,2.16,0.73,0.45


In [5]:
dataset.columns

Index(['min_sip', 'min_lumpsum', 'expense_ratio', 'fund_size_cr',
       'fund_age_yr', 'risk_level', 'rating', 'returns_1yr', 'returns_3yr',
       'returns_5yr', 'sortino', 'alpha', 'sd', 'beta', 'sharpe'],
      dtype='object')

In [6]:
indep_X=dataset[['min_sip', 'min_lumpsum', 'fund_size_cr',
       'fund_age_yr', 'risk_level', 'rating', 'returns_1yr', 'returns_3yr',
       'returns_5yr']]

In [7]:
dep_Y=dataset[['expense_ratio']]

In [8]:
#Recursive Feature Elimination

In [9]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)

        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [10]:
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

In [11]:
def Linear(X_train,y_train,X_test):       
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

In [12]:
def Decision(X_train,y_train,X_test):
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

In [13]:
def random(X_train,y_train,X_test):       
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [14]:
def xgboost(X_train,y_train,X_test):       
        from xgboost import XGBRegressor
        regressor = XGBRegressor(n_jobs=5,learning_rate=0.1,max_depth=10,random_state=1)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2

In [15]:
n=10

In [16]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

def split_scalar(X, y, test_size=0.2, random_state=0):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def r2_prediction(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return np.corrcoef(y_test, y_pred)[0, 1] ** 2

def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    colnames_list = []
    r2_values = []

    lin = LinearRegression()
    dec = DecisionTreeRegressor(random_state=0)
    rf = RandomForestRegressor(n_estimators=100, random_state=0)  # Increase the number of trees
    xgb = XGBRegressor(n_jobs=5, learning_rate=0.05, n_estimators=300, max_depth=10, random_state=1)  # Adjust learning rate and estimators

    rfemodellist = [lin, dec, rf, xgb]

    for model in rfemodellist:
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y.values.ravel())  # Use .ravel() to avoid the warning
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)

        selected_columns = [col for col, selected in zip(indep_X.columns, log_rfe.support_) if selected]
        colnames_list.append(selected_columns)

        X_train, X_test, y_train, y_test = split_scalar(pd.DataFrame(log_rfe_feature), dep_Y)

        # Use cross-validation to get a robust estimate of model performance
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
        r2 = scores.mean()
        r2_values.append(r2)

    return rfelist, colnames_list, r2_values

# Example usage:
# Assuming you have a dataframe `df` with independent variables in `X` and dependent variable in `y`
# X = df.drop('target', axis=1)
# y = df['target']
# features, selected_columns, r2_scores = rfeFeature(X, y, 10)
# print(selected_columns)
# print(r2_scores)


In [17]:
rfeFeature(indep_X,dep_Y,n)

C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

([array([[1.000e+02, 1.000e+02, 1.000e+01, ..., 4.000e+00, 6.500e+00,
          6.900e+00],
         [1.000e+03, 1.000e+03, 4.288e+03, ..., 5.600e+00, 4.800e+00,
          5.500e+00],
         [1.000e+03, 1.000e+03, 1.570e+02, ..., 2.000e+00, 1.890e+01,
          9.700e+00],
         ...,
         [5.000e+02, 5.000e+02, 3.330e+02, ..., 3.800e+00, 6.100e+00,
          4.900e+00],
         [5.000e+02, 5.000e+02, 2.065e+03, ..., 2.600e+00, 5.600e+00,
          0.000e+00],
         [5.000e+02, 5.000e+02, 2.100e+02, ..., 1.080e+01, 6.600e+00,
          4.500e+00]]),
  array([[1.000e+02, 1.000e+02, 1.000e+01, ..., 4.000e+00, 6.500e+00,
          6.900e+00],
         [1.000e+03, 1.000e+03, 4.288e+03, ..., 5.600e+00, 4.800e+00,
          5.500e+00],
         [1.000e+03, 1.000e+03, 1.570e+02, ..., 2.000e+00, 1.890e+01,
          9.700e+00],
         ...,
         [5.000e+02, 5.000e+02, 3.330e+02, ..., 3.800e+00, 6.100e+00,
          4.900e+00],
         [5.000e+02, 5.000e+02, 2.065e+03, ..., 2.

In [18]:
# Call the function with your data
rfelist, colnames_list, r2_values = rfeFeature(indep_X, dep_Y,200)

C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

In [19]:
# Print the selected column names and R2 values for each model
for model_name, selected_columns, r2_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, r2_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"R2 Value: {r2_value}\n")

Model: Linear
Selected Columns: ['min_sip', 'min_lumpsum', 'fund_size_cr', 'fund_age_yr', 'risk_level', 'rating', 'returns_1yr', 'returns_3yr', 'returns_5yr']
R2 Value: 0.2729252353289947

Model: Decision
Selected Columns: ['min_sip', 'min_lumpsum', 'fund_size_cr', 'fund_age_yr', 'risk_level', 'rating', 'returns_1yr', 'returns_3yr', 'returns_5yr']
R2 Value: 0.20600916347396936

Model: Random
Selected Columns: ['min_sip', 'min_lumpsum', 'fund_size_cr', 'fund_age_yr', 'risk_level', 'rating', 'returns_1yr', 'returns_3yr', 'returns_5yr']
R2 Value: 0.5940347089143893

Model: XGBoost
Selected Columns: ['min_sip', 'min_lumpsum', 'fund_size_cr', 'fund_age_yr', 'risk_level', 'rating', 'returns_1yr', 'returns_3yr', 'returns_5yr']
R2 Value: 0.5128810678575657



# Model Creation

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep_X, dep_Y, test_size=0.30, random_state=0)
from sklearn.ensemble import RandomForestRegressor
# Instantiate the RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=10000, criterion="friedman_mse", max_samples=None, bootstrap=True, min_impurity_decrease=0.0, min_samples_split=2, max_features="auto", oob_score=True)
regressor=regressor.fit(x_train,y_train)

C:\Users\ragus\AppData\Local\Temp\ipykernel_2980\4092912710.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor=regressor.fit(x_train,y_train)
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [21]:
y_pred=regressor.predict(x_test)

In [22]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)

In [23]:
r_score

0.40685950231269963

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep_X,dep_Y, test_size=0.30, random_state=0)

In [25]:
from sklearn.preprocessing import StandardScaler 
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [26]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor
param_grid={'criterion':['squared_error','squared_error'],'max_features':['auto','sqrt','log2'],'n_estimators':[100]}
grid=GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)

In [27]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'squared_error'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100]},
             verbose=3)

In [28]:
y_pred=grid.predict(x_test)

In [29]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.420720203069994

In [30]:
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeRegressor
param_grid={'criterion':['mse','mae','friedman_mse'],'max_features':['auto','sqrt','log2'],'splitter':['best','random']}

In [31]:
grid_Decision=GridSearchCV(DecisionTreeRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid_Decision.fit(indep_X,dep_Y)

C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\base.py", line 6

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
 -0.18233423 -0.45589793 -0.3996853  -0.69518818 -0.23735555 -0.52075838]
  warnings.warn(
C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae', 'friedman_mse'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [32]:
y_pred=grid_Decision.predict(x_test)

C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [33]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

-0.1335824583390648

In [34]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_Grid={"kernel":["rbf","poly","sigmoid","linear"],"C":[10,200,300,1000,2000],"gamma":["auto","scale"]}
Grid=GridSearchCV(SVR(),param_Grid,refit=True,verbose=3,n_jobs=-1)

In [36]:
Grid.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [10, 200, 300, 1000, 2000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             verbose=3)

In [37]:
y_pred=Grid.predict(x_test)

In [38]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.3187202455470426

In [44]:
from sklearn.ensemble import AdaBoostRegressor
regressor_ada=AdaBoostRegressor(n_estimators=50, learning_rate=1.0, random_state=42)
regressor_ada.fit(x_train,y_train)

C:\Users\ragus\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(random_state=42)

In [45]:
y_pred=regressor_ada.predict(x_test)

In [46]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.4289028536344168

In [48]:
from lightgbm import LGBMRegressor
regressor_LGM=LGBMRegressor(num_threads=2,learning_rate=0.05,num_iterations=100,max_depth=100,colsample_bytree=0.8,reg_alpha=1e-2,reg_lambda=1e-2,seed=42)
regressor_LGM.fit(x_train,y_train.values.ravel())

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000023 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 159
[LightGBM] [Info] Number of data points in the train set: 104, number of used features: 9
[LightGBM] [Info] Start training from score 0.662019
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

C:\Users\ragus\anaconda3\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.05, max_depth=100,
              num_iterations=100, num_threads=2, reg_alpha=0.01,
              reg_lambda=0.01, seed=42)

In [49]:
y_pred=regressor_LGM.predict(x_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [50]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.3750022864824677

In [55]:
from xgboost import XGBRegressor
regressor=xgb.XGBRegressor(learning_rate=0.1,n_estimators=100,max_depth=3,subsample=0.8,colsample_bytree=0.8,reg_alpha=1e-2, reg_lambda=1e-2,random_state=42)
regressor.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [56]:
 y_pred=regressor.predict(x_test)

In [57]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.3959719268478403

In [79]:
re=grid.cv_results_
print=("best_parameter{}:".format(grid.best_params_))

In [80]:
table=pd.DataFrame.from_dict(re)

In [81]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.176162,0.005136,0.006262,0.007670,squared_error,auto,100,"{'criterion': 'squared_error', 'max_features':...",0.553321,0.610690,0.647842,0.430899,0.381367,0.524824,0.102684,5
1,0.168749,0.016910,0.009111,0.008170,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.511617,0.685760,0.719673,0.552160,0.378953,0.569632,0.123310,1
2,0.173644,0.031166,0.006240,0.007643,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.510403,0.591196,0.704358,0.546420,0.363011,0.543078,0.111202,4
3,0.197742,0.015309,0.003125,0.006250,squared_error,auto,100,"{'criterion': 'squared_error', 'max_features':...",0.533227,0.597424,0.675416,0.463852,0.351663,0.524317,0.111104,6
4,0.152393,0.011129,0.012502,0.006251,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.500511,0.614622,0.734274,0.579450,0.299220,0.545616,0.144391,3
5,0.123764,0.011692,0.003125,0.006250,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.525160,0.638762,0.727983,0.582283,0.347375,0.564313,0.127474,2


In [82]:
min_sip = int(input("Enter min_sip: "))
min_lumpsum = int(input("Enter min_lumpsum: "))
fund_size_cr = int(input("Enter fund_size_cr: "))
fund_age_yr = int(input("Enter fund_age_yr: "))
risk_level = int(input("Enter risk_level: "))
rating = int(input("Enter rating: "))
returns_1yr = float(input("Enter returns_1yr: "))
returns_3yr = float(input("Enter returns_3yr: "))
returns_5yr = float(input("Enter returns_5yr: "))

Enter min_sip:  100
Enter min_lumpsum:  100
Enter fund_size_cr:  23
Enter fund_age_yr:  10
Enter risk_level:  3
Enter rating:  3
Enter returns_1yr:  4
Enter returns_3yr:  6
Enter returns_5yr:  6


In [83]:
#Make sure the grid search is properly fitted
regressor.fit(x_train,y_train)
Future_Prediction=grid.predict([[min_sip,min_lumpsum,fund_size_cr,fund_age_yr,risk_level,rating,returns_1yr,returns_3yr,returns_5yr]])
print=("Future_prediction={}".format(Future_Prediction))

In [87]:
print

'Future_prediction=[1.1511]'

In [88]:
# comparing, Decision Tree algorithm gives maximum accuracy of 0.14%

In [89]:
import pickle
Finalised_Model="Finalized_model.sav"

In [90]:
pickle.dump(regressor,open(Finalised_Model,'wb'))